# Test Your Algorithm

## Instructions
1. From the **Pulse Rate Algorithm** Notebook you can do one of the following:
   - Copy over all the **Code** section to the following Code block.
   - Download as a Python (`.py`) and copy the code to the following Code block.
2. In the bottom right, click the <span style="color:blue">Test Run</span> button. 

### Didn't Pass
If your code didn't pass the test, go back to the previous Concept or to your local setup and continue iterating on your algorithm and try to bring your training error down before testing again.

### Pass
If your code passes the test, complete the following! You **must** include a screenshot of your code and the Test being **Passed**. Here is what the starter filler code looks like when the test is run and should be similar. A passed test will include in the notebook a green outline plus a box with **Test passed:** and in the Results bar at the bottom the progress bar will be at 100% plus a checkmark with **All cells passed**.
![Example](example.png)

1. Take a screenshot of your code passing the test, make sure it is in the format `.png`. If not a `.png` image, you will have to edit the Markdown render the image after Step 3. Here is an example of what the `passed.png` would look like 
2. Upload the screenshot to the same folder or directory as this jupyter notebook.
3. Rename the screenshot to `passed.png` and it should show up below.
![Passed](passed.png)
4. Download this jupyter notebook as a `.pdf` file. 
5. Continue to Part 2 of the Project. 

In [ ]:
"""
This algorithm calculates heart beats estimates along with the confidence of each estimate
from a dataset containing ppg and 3-axis accelerometer signals.
The evaluation of the performance is made by computing the mean absolute error (MAE)
of the 90% most confident estimates.
"""
import glob

import numpy as np
import scipy as sp
import scipy.io
import scipy.signal

import matplotlib.pyplot as plt  # look at the data


def LoadTroikaDataset():
    """
    Retrieve the .mat filenames for the troika dataset.

    Review the README in ./datasets/troika/ to understand the organization of the .mat files.

    Returns:
        data_fls: Names of the .mat files that contain signal data
        ref_fls: Names of the .mat files that contain reference data
        <data_fls> and <ref_fls> are ordered correspondingly, so that ref_fls[5] is the 
            reference data for data_fls[5], etc...
    """
    data_dir = "./datasets/troika/training_data"
    data_fls = sorted(glob.glob(data_dir + "/DATA_*.mat"))
    ref_fls = sorted(glob.glob(data_dir + "/REF_*.mat"))
    return data_fls, ref_fls


def LoadTroikaDataFile(data_fl):
    """
    Loads and extracts signals from a troika data file.

    Usage:
        data_fls, ref_fls = LoadTroikaDataset()
        ppg, accx, accy, accz = LoadTroikaDataFile(data_fls[0])

    Args:
        data_fl: (str) filepath to a troika .mat file.

    Returns:
        numpy arrays for ppg, accx, accy, accz signals.
    """
    data = sp.io.loadmat(data_fl)['sig']
    return data[2:]


def AggregateErrorMetric(pr_errors, confidence_est):
    """
    Computes an aggregate error metric based on confidence estimates.

    Computes the MAE at 90% availability. 

    Args:
        pr_errors: a numpy array of errors between pulse rate estimates and corresponding 
            reference heart rates.
        confidence_est: a numpy array of confidence estimates for each pulse rate
            error.

    Returns:
        the MAE at 90% availability
    """
    # Higher confidence means a better estimate. The best 90% of the estimates
    #    are above the 10th percentile confidence.
    percentile90_confidence = np.percentile(confidence_est, 10)

    # Find the errors of the best pulse rate estimates
    best_estimates = pr_errors[confidence_est >= percentile90_confidence]

    # Return the mean absolute error
    return np.mean(np.abs(best_estimates))


def Evaluate():
    """
    Top-level function evaluation function.

    Runs the pulse rate algorithm on the Troika dataset and returns an aggregate error metric.

    Returns:
        Pulse rate error on the Troika dataset. See AggregateErrorMetric.
    """
    # Retrieve dataset files
    data_fls, ref_fls = LoadTroikaDataset()
    errs, confs = [], []
    for data_fl, ref_fl in zip(data_fls, ref_fls):
        # Run the pulse rate algorithm on each trial in the dataset
        errors, confidence = RunPulseRateAlgorithm(data_fl, ref_fl)
        errs.append(errors)
        confs.append(confidence)
    # Compute aggregate error metric
    errs = np.hstack(errs)
    confs = np.hstack(confs)
    return AggregateErrorMetric(errs, confs)


def RunPulseRateAlgorithm(data_fl, ref_fl):
    """
    Main function to estimate the heart rates with a confidence factor and compare them to the ground truth.

    Args:
        data_fl: (str) filepath to a troika .mat file containing the ppg and accelerometer signals.
        ref_fl: (str) filepath to a troika .mat file containing the ground truth estimates.

    Returns:
        Per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    """
    # Load data using LoadTroikaDataFile
    ppg, accx, accy, accz = LoadTroikaDataFile(data_fl)
    
    # Bandpass all the signals
    ppg = BandpassFilterBPM(ppg)
    accx = BandpassFilterBPM(accx)
    accy = BandpassFilterBPM(accy)
    accz = BandpassFilterBPM(accz)
    
    # Retrieve the reference heart rates
    ref = sp.io.loadmat(ref_fl)['BPM0']
    
    # Compute pulse rate estimates and estimation confidence.
    errors = []
    confidence = []
    
    # sliding window: width 8s, shift 2s
    fs = 125  # according to the documentation, the sampling frequency is 125Hz
    win_width = 8 * fs  # The width & overlap correspond to how the reference BPM was calculated
    win_overlap = 6 * fs

    # Find dominant frequencies for the accelerometer data
    acc_sign = BandpassFilterBPM(AccelerometerMagnitude(accx, accy, accz))
    acc_spec, acc_freqs, _, _ = plt.specgram(acc_sign, NFFT=win_width, Fs=fs, noverlap=win_overlap)
    acc_dominant_freqs = acc_freqs[np.argmax(acc_spec, axis=0)] * 60  # convert to BPM

    # Define the width of a peak and the number of frequencies to consider per estimate
    epsilon = 0  # A peak as a unique frequency (epsilon=0) grants the best results
    n_best_freqs = 7  # This parameter is also defined by trial and error

    # Calculate the spectogram of the ppg signal on the previously defined windows
    ppg_spec, ppg_freqs, _, _ = plt.specgram(ppg, NFFT=win_width, Fs=fs, noverlap=win_overlap, detrend='linear')

    # Calculate the heart rate estimates
    hr_estimates = np.array([])
    for i in range(len(ppg_spec[0,])):
        hr_estimates = np.append(hr_estimates, SmoothEstimates(ppg_spec, ppg_freqs, hr_estimates, n_best_freqs))
    
    # Try to separate arm motion noise from the actual heart beat frequencies
    hr_estimates = AvoidArmMotionNoise(hr_estimates, acc_dominant_freqs, ppg_spec, ppg_freqs, n_best_freqs, epsilon)

    # Calculate error and confidence
    errors = np.absolute(hr_estimates - ref.flatten())  # we calculated our estimates in the same windows as the reference
    confidence = np.array([])
    for i, estimate in enumerate(hr_estimates):
        confidence = np.append(confidence, CalcConfidence(estimate, ppg_spec[:, i], ppg_freqs, epsilon))

    # Return per-estimate mean absolute error and confidence as a 2-tuple of numpy arrays.
    return errors, confidence


def BandpassFilterBPM(signal, fs=125, lower_BPM=40, higher_BPM=240):
    """
    Bandpass filter the signal between 40 & 240 BPM by default.
    
    Args:
        signal: (array) signal to bandpass.
        fs: (int) sampling frequency of the signal, default is 125.
        lower_BPM: (int) lower frequency to bandpass the filter with in BPM, default is 40.
        higher_BPM: (int) higher frequency to bandpass the filter with in BPM, default is 240.

    Returns:
        Filtered signal in a numpy array.
    """
    b, a = sp.signal.butter(3, (lower_BPM/60, higher_BPM/60), btype='bandpass', fs=fs)
    return sp.signal.filtfilt(b, a, signal)


def AccelerometerMagnitude(accx, accy, accz):
    """
    Aggregate the accelerometer signals into a magnitude signal.
    
    Args:
        accx: (array) accelerometer signal on the x axis.
        accy: (array) accelerometer signal on the y axis.
        accz: (array) accelerometer signal on the z axis.

    Returns:
        Aggregated signal in a numpy array.
    """
    return np.sqrt(np.square(accx) + np.square(accy) + np.square(accz))


def SmoothEstimates(ppg_spec, ppg_freqs, hr_estimates, n_best_freqs):
    """
    Calculate a heart rate estimate from the spectrum of a fraction of a signal
        while taking into account previous estimates.
    
    Args:
        ppg_spec: (array) spectogram of the ppg signal the estimates are made on.
        ppg_freqs: (array) frequencies used to calculate the spectogram of the ppg signal.
        hr_estimates: (array) array containing the previous heart rate estimates.
        n_best_freqs: (int) number of the best frequencies to consider for the estimate.
            The closest of those frequencies to the previous estimate is chosen.

    Returns:
        An estimate in BPM of the heart rate frequency for the given spectrum.
    """
    # retrieve the index of the current heart rate estimate
    index = len(hr_estimates)
    # if the current estimate is not the first, we take into account the previous one
    if index != 0:
        last_estimate = hr_estimates[-1]/60
        # First, we retrieve the most impactful n_best_freqs frequencies on the spectrum
        candidates = np.argsort(ppg_spec[:, index], axis=0)[-n_best_freqs:]
        # Then, the estimate is the closest of those candidates to the last estimate to avoid unrealistic sudden shifts
        estimate = ppg_freqs[candidates[np.argmin(np.absolute(ppg_freqs[candidates] - last_estimate))]] * 60
    else:
        estimate = ppg_freqs[np.argmax(ppg_spec[:,0], axis=0)] * 60
    return estimate


def AvoidArmMotionNoise(hr_estimates, acc_dominant_freqs, ppg_spec, ppg_freqs, n_best_freqs, epsilon=0):
    """
    Try to detect and reduce the potential noise of the arm motion in the calculated heart rate estimates.
    
    Args:
        hr_estimates: (array) array containing all of the heart rate estimates.
        acc_dominant_freqs: (array) array containing the dominant frequencies
            in the aggregated accelerometer signal.
        ppg_spec: (array) spectogram of the ppg signal the estimates were made on.
        ppg_freqs: (array) frequencies used to calculate the spectogram of the ppg signal.
        n_best_freqs: (int) number of the best frequencies to consider for each estimate.
            The closest of those frequencies to the previous estimate is chosen.
        epsilon: (int) corresponds to half of the width of a peak in number of BPMs
            (if epsilon=0, a peak is a unique frequency).      

    Returns:
        A potentially modified array of heart rate estimates with less noise from arm movements.
    """
    # if a hr estimate is close to an accelerometer dominant frequency, then try to find other candidates
    noisy_samples = np.where(np.absolute(hr_estimates-acc_dominant_freqs) <= epsilon )[0]

    for noisy_sample in noisy_samples:
        # if the first estimate is potentially noisy, we do not modify it as we need previous estimates
        if (noisy_sample == 0):
            continue

        # retrieve the last and current estimates
        last_estimate = hr_estimates[noisy_sample-1]/60
        current_estimate = hr_estimates[noisy_sample]/60

        # the new candidates are samples whose distance to the acc dominant frequency is more than epsilon
        candidates = np.where((np.absolute((ppg_freqs*60) - acc_dominant_freqs[noisy_sample]) > epsilon))
        
        if len(candidates[0]) != 0:
            # we are looking for the closest freq among the candidate frequencies with the highest amplitude
            best_candidates = np.argsort(ppg_spec[candidates, noisy_sample])[0][-n_best_freqs:]
            closest_candidates = best_candidates[np.argsort(np.absolute(ppg_freqs[best_candidates] - last_estimate))]

            # Here are the rules to define a good candidate:
            # 1. If the closest candidate implies a bigger or equal shift than the current shift, it is discarded
            # This is because the smooth estimates calculation already reduces the risk of noise
            # Avoiding a potential noise shouldn't lead to a less realistic evolution or even a huge shift
            if (np.absolute(ppg_freqs[closest_candidates[0]] - last_estimate) >=
                np.absolute(current_estimate - last_estimate)):
                continue

            # 2. A good candidate is at least half of the potentially noisy frequency
            # We try to find one such candidate starting from the closest to the furthest
            for i in range(n_best_freqs):
                if (2 * ppg_spec[closest_candidates[i], noisy_sample] >= acc_dominant_freqs[noisy_sample]):
                    hr_estimates[noisy_sample] = ppg_freqs[best_candidates[i]]*60
                    break

    return hr_estimates


def CalcConfidence(hr_bpm, fft, freqs, epsilon=0):
    """
    This modified SNR algorithm calculates the confidence we have in one estimated heart rate.

    Args:
        hr_bpm: (int) heart rate estimation in BPM to calculate the confidence of.
        fft: (array) spectogram of the signal the estimate was made on.
        freqs: (array) frequencies used to calculate the spectogram.
        epsilon: (int) corresponds to half of the width of a peak in number of BPMs
            (if epsilon=0, the peak is a unique frequency).

    Returns:
        numpy float for the signal to noise ratio which is our confidence value.
    """
    # BPM to Hz
    epsilon = epsilon/60
    hr_freq = hr_bpm/60

    # Find the frequencies to compare to the entire spectrum
    hr_borders = (hr_freq-epsilon, hr_freq+epsilon)
    hr_index = np.where((freqs>=hr_borders[0]) & (freqs<=hr_borders[1]))

    # Compute signal power and noise power
    signal_power = np.sum(np.abs(fft[hr_index]))
    noise_power = np.sum(np.abs(fft))

    # Compute SNR
    snr = signal_power / noise_power
    return snr